In [2]:
import pandas as pd


In [3]:
df = pd.read_csv("../data/raw/small_data_1gb.csv")   # adjust path if needed

print("Initial dataset info:")
df.info()
display(df.head())

Initial dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26132955 entries, 0 to 26132954
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   info       object 
 1   room_id    object 
 2   date_time  object 
 3   bool_v     object 
 4   str_v      object 
 5   long_v     float64
 6   dbl_v      float64
dtypes: float64(2), object(5)
memory usage: 1.4+ GB


,info,room_id,date_time,bool_v,str_v,long_v,dbl_v
0,light,6.315B,2019-02-09 16:52:53,NaN,NaN,0.0,NaN
1,temp,6.107,2019-02-09 16:52:55,NaN,NaN,NaN,27.55
2,co2,6.107,2019-02-09 16:52:55,NaN,NaN,55.0,NaN
3,humidity,6.225,2019-02-09 16:52:58,NaN,NaN,43.0,NaN
4,rssi,6.225,2019-02-09 16:52:58,NaN,NaN,-104.0,NaN


In [4]:
# -----------------------------
# Step 2: Inspect unique sensor types
# -----------------------------
counts = df["info"].value_counts()
print("\nSensor type counts:")
print(counts)


Sensor type counts:
info
co2         2010241
light       2010236
snr         2010235
humidity    2010233
orient      2010233
pressure    2010232
checksum    2010227
temp        2010226
roomNum     2010226
movement    2010219
rssi        2010218
voltage     2010218
nodeId      2010211
Name: count, dtype: int64


In [5]:
# -----------------------------
# Step 3: Unify value columns
# -----------------------------
df["value"] = (
    df["bool_v"].combine_first(df["long_v"])
                .combine_first(df["dbl_v"])
                .combine_first(df["str_v"])
)

df = df.drop(columns=["bool_v", "long_v", "dbl_v", "str_v"])

In [6]:
# -----------------------------
# Step 4: Pivot to wide format
# -----------------------------
wide = df.pivot_table(
    index=["date_time", "room_id"],
    columns="info",
    values="value",
    aggfunc="first"
).reset_index()

# Remove unreliable features
wide = wide.drop(columns=["checksum", "orient"], errors="ignore")

print("\nWide-format dataset:")
display(wide.head())


Wide-format dataset:


info,date_time,room_id,co2,humidity,light,movement,nodeId,pressure,roomNum,rssi,snr,temp,voltage
0,2019-02-09 16:52:53,6.315B,NaN,NaN,0.0,NaN,NaN,NaN,NaN,-104.0,NaN,NaN,4.24
1,2019-02-09 16:52:55,6.107,55.0,43.0,NaN,NaN,5CABAE0D,NaN,NaN,-86.0,NaN,27.55,NaN
2,2019-02-09 16:52:57,6.315F,NaN,NaN,NaN,NaN,NaN,993.3,6.315F,NaN,10.7,NaN,4.24
3,2019-02-09 16:52:58,6.225,NaN,43.0,NaN,NaN,NaN,NaN,NaN,-104.0,NaN,26.66,NaN
4,2019-02-09 16:52:59,6.306,NaN,NaN,NaN,False,NaN,993.1,6.306,NaN,12.0,NaN,4.24


In [7]:
# -----------------------------
# Step 5: Null value report
# -----------------------------
null_report = wide.isnull().sum().to_frame("null_count")
null_report["percent"] = (null_report["null_count"] / len(wide)) * 100
print("\nNull value report:")
print(null_report)



Null value report:
           null_count   percent
info                           
date_time           0  0.000000
room_id             0  0.000000
co2               189  0.009401
humidity          197  0.009799
light             194  0.009650
movement          211  0.010495
nodeId            219  0.010893
pressure          198  0.009849
roomNum           204  0.010147
rssi              212  0.010545
snr               195  0.009699
temp              204  0.010147
voltage           212  0.010545


In [9]:
# -----------------------------
# Step 6: Drop rows with nulls
# -----------------------------
wide_clean = wide.dropna()
print(f"\nAfter cleaning: {len(wide_clean)} rows")

# Ensure object columns are strings
for col in wide_clean.select_dtypes(include="object").columns:
    wide_clean[col] = wide_clean[col].astype(str)

display(wide_clean.head())


After cleaning: 2010077 rows


/var/folders/hb/qtfhw3pd1wq6wpvfwpr8dqt40000gn/T/ipykernel_95737/3353481363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide_clean[col] = wide_clean[col].astype(str)
/var/folders/hb/qtfhw3pd1wq6wpvfwpr8dqt40000gn/T/ipykernel_95737/3353481363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide_clean[col] = wide_clean[col].astype(str)
/var/folders/hb/qtfhw3pd1wq6wpvfwpr8dqt40000gn/T/ipykernel_95737/3353481363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

info,date_time,room_id,co2,humidity,light,movement,nodeId,pressure,roomNum,rssi,snr,temp,voltage
40,2019-02-09 16:53:21,6.G09,55.0,71.0,0.0,True,5DAB315B,994.9,6.G09,-108.0,12.7,23.44,4.26
55,2019-02-09 16:53:29,6.217,55.0,34.0,1.0,False,9CAB3AA5,993.7,6.217,-79.0,10.7,27.8,4.18
64,2019-02-09 16:53:34,6.226,55.0,49.0,1.0,True,25AB5127,993.8,6.226,-111.0,11.2,27.59,4.22
69,2019-02-09 16:53:39,6.199C,55.0,49.0,1.0,False,57AB8D01,994.4,6.199C,-90.0,11.2,25.78,4.24
70,2019-02-09 16:53:40,6.110B,55.0,53.0,0.0,False,7EAB4682,994.7,6.110B,-117.0,11.5,25.38,4.0


In [11]:
# -----------------------------
# Step 7: Save cleaned data
# -----------------------------
wide_clean.to_parquet(
    "../data/processed/building_replay",
    engine="fastparquet",
    compression="snappy",
    partition_cols=["room_id"]
)
print("\n[Done] Cleaned dataset saved to ../data/processed/building_replay/")



[Done] Cleaned dataset saved to ../data/processed/building_replay/
